In [9]:
import codecs
import io
import numpy as np
import re
import sys
import os
import time
import jaconv
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from matplotlib import pyplot as plt, font_manager
from wordcloud import WordCloud
from janome.tokenizer import Tokenizer

font_manager._rebuild()
if os.name=='nt':
    #windows用
    font_dir=font_manager.win32FontDirectory()
else:
    #mac用
    font_dir='/Users/pydata/Library/Fonts/'
font_path=os.path.join(font_dir,'SourceHanCodeJP-Regular.otf')
font=font_manager.FontProperties(fname=font_path,size=14)

cps=[]

class CorpusElement:
    def __init__(self,text='',tokens=[],pn_scores=[]):
        self.text=text
        self.tokens=tokens
        self.pn_scores=pn_scores
        
def load_pn_dict():
    dic={}
    
    with codecs.open('./pn_ja.dic','r','utf-8') as f:
        lines=f.readlines()
        
        for line in lines:
            columns=line.split(':')
            dic[columns[0]]=float(columns[3])
    
    return dic

def get_pn_scores(tokens, pn_dic):
    scores=[]
    
    for surface in [t.surface for t in tokens if t.part_of_speech.split(',')[0] in ['動詞', '名詞','形容詞', '副詞']]:
        if surface in pn_dic:
            scores.append(pn_dic[surface])
    
    return scores
        
def read_file(filename='Roselia\\BLACK SHOUT.txt'):
    with open(filename, 'r', encoding='utf-8') as f:
        text=f.read()
#     text=re.sub('\n','',text)
    return text

def dict_list(text):
    ja_tokenizer = Tokenizer()
    text=re.sub('-','',text)
    text=re.sub('"','',text) 
    lines = text.split('\n')
    diclist = []
    for text in lines:
        tokens = ja_tokenizer.tokenize(text)
        element = CorpusElement(text, tokens)
        cps.append(element)

def output(data,filename):
    with open(filename + '.csv', 'w', newline='',encoding='utf-8') as f:
        f.write('コメント,マネポジ値\n')
        for key, value in data.items():
            f.write('{},{}\n'.format(key,value))
    f.close()

if __name__ == "__main__":
    pn_dic=load_pn_dict()
    text=read_file()
    dict_list(text)
    out_dic={}
    
    i = 0
    for element in cps:
        element.pn_scores = get_pn_scores(element.tokens, pn_dic)
        if len(cps[i].pn_scores) != 0:
            out_dic[io.StringIO(element.text).readline()] = float(sum(element.pn_scores)/float(len(cps[i].pn_scores)))
        i += 1
#     print(out_dic)
    output(out_dic,'negaposi')
    print('ave:',sum(out_dic.values())/len(out_dic))    
    print('max:',max(out_dic),max(out_dic.values()))
    print('min:',min(out_dic),min(out_dic.values()))
#     pass

ave: -0.6624297784313726
max: 震えた胸【凛と】 -0.19670499999999996
min: 【BLACK】モノクロの雨が -0.9812825000000001
